**Rationale** This assignment will help you practice running and interpreting basic regressions analysis.

**[Datasets](https://drive.google.com/drive/folders/1LJ38ctVX62ggGGygJDLlLG2V6jZ4BNml?usp=sharing)**
1. [Safegraph Temple Visitor Sample](https://drive.google.com/file/d/1u_sDxFp9fja-_cop4yzIAJ_hX2-fplWN/view?usp=sharing) 
1. [Avocado Prices](https://drive.google.com/file/d/1QJ_dyyBFkHhC6VkzVvdAFa385mtqtntM/view?usp=sharing)

**Data description** This dataset includes SafeGraph's sampled foot traffic volume for Temple University between 1-1-2019 and 5-31-2020. Safegraph collects the foot traffic data from a network of phone apps. You can view the `visits` data as the number of SafeGraph tracked devices to visit Temple University. This dataset is merged with Philadelphia's weather data. 


# 1. Run and interpret a simple linear regression (5 points)...
...to figure out the relationship between daily average temperateure as measured in degrees fahrenheit, `"temp"`, and people on campus `"visits"` for the Fall 2019 semester (limit data to M-F 8-26 - 11-24-2019, inclusive). 

In your answer: 
1. First, look at data. What is the average number of SafeGraph visitors to Temple University during the teaching days of the Fall 2019 semester? Roughly what percent of the Temple student population is this? **138**
1. Specify (write out the regression equation), run, and interpret a regression that documents the relationship between temperature and foot traffic (visits) on campus. **36%**
    1. Do more students visit campus (and presumably go to class) when the weather is warmer or colder? **warmer**
    1. Besides students not wanting to go to class when the temperature is not to their liking, what else might cause the observed relationship between visits and temperature? **transportation, traffic**

In [3]:
# imports here
import os, pandas as pd, matplotlib.pyplot as plt
from google.colab import drive
from statsmodels.formula import api as smf
drive.mount('drive')

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Mounted at drive


In [4]:
# list directory containing the data
fpath = 'drive/MyDrive/Customer Data Analysis/' # change this to your data folder
os.listdir(fpath)

['MyNotes Mod1.ipynb',
 'A1 Treasure Love_Part1.ipynb',
 'A1_Part2.ipynb',
 'MyNotes_Mod2.ipynb',
 'A2 Programming Fundamentals.ipynb',
 'stores A data-set.csv',
 'stores data-set.csv',
 'stores B data-set.csv',
 'stores C data-set.csv',
 'sales data-set.csv',
 '9-8_notes2.ipynb',
 'Colab Notebooks',
 'quebec_yelp_reviews.csv',
 'quebec_biz.csv',
 'Quebec_dataset.csv',
 '9-8_notes.ipynb',
 'A3.ipynb',
 'Copy of InClass_Module_3.ipynb',
 'telecom_churn.csv',
 'renttherunway_final_data.json',
 'A4.ipynb',
 'Notes_Sept15.ipynb',
 '9 22_Notes.ipynb',
 '9-29_notes.ipynb',
 'facebook_ads.csv',
 'starbucks_promos.csv',
 'A4 F2021 - Visualization.ipynb',
 'A5.ipynb',
 '10 13_notes.ipynb',
 'temple_traffic.csv',
 'avocado.csv']

In [7]:
# read in the dataset
df = pd.read_csv(fpath + 'temple_traffic.csv', index_col = 0)

In [8]:
# convert the date column to pandas datetime using pd.to_datetime(...)
# if you've fogotten, look through notes from module 3

df.date = pd.to_datetime(df.date)

In [9]:
# create a new column called 'dow' (for day of week) using temple.date.dt.weekday
# note that Monday through Sunday corresponds to 0 through 6
df['dow'] = df.date.dt.weekday


In [10]:
# select only the rows where the date lies between 2019/8/26 and 2019/11/24 (inclusive) 
# and correspond to a regular school date (e.g. are M-F)
# call the reuslting dataframe F2019
# remember you can select rows using df.loc[conditions]
# If you forgot how to do this, look at the notes and in class exercise from module 3

Y2019 = df.loc[(df.date>='2019/08/26')&(df.date<'2019/11/24')&(df.dow.isin([0,1,2,3,4]))]


In [37]:
Y2019.head()

,safegraph_place_id,date,visits,stn,wban,temp,count_temp,dewp,count_dewp,slp,count_slp,stp,count_stp,visib,count_visib,wdsp,count_wdsp,mxspd,gust,maxtemp,flag_max,mintemp,flag_min,prcp,flag_prcp,sndp,frshtt,dow
7,sg:00579e722c8e48178ef0c66a7c91f92c,2019-09-23,169,997286.0,99999.0,80.2,23.0,NaN,0.0,1012.9,23.0,NaN,0.0,NaN,0.0,0.0,23.0,NaN,NaN,90.7,*,70.9,*,0.0,I,NaN,0.0,0
8,sg:00579e722c8e48178ef0c66a7c91f92c,2019-09-24,182,997286.0,99999.0,75.3,24.0,NaN,0.0,1009.2,24.0,NaN,0.0,NaN,0.0,0.0,24.0,NaN,NaN,83.3,*,66.7,*,0.0,I,NaN,0.0,1
9,sg:00579e722c8e48178ef0c66a7c91f92c,2019-09-25,156,997286.0,99999.0,72.3,24.0,NaN,0.0,1011.7,24.0,NaN,0.0,NaN,0.0,0.0,24.0,NaN,NaN,81.0,*,61.7,*,0.0,I,NaN,0.0,2
10,sg:00579e722c8e48178ef0c66a7c91f92c,2019-09-26,168,997286.0,99999.0,72.1,24.0,NaN,0.0,1009.5,24.0,NaN,0.0,NaN,0.0,0.0,24.0,NaN,NaN,87.3,*,63.5,*,0.0,I,NaN,0.0,3
11,sg:00579e722c8e48178ef0c66a7c91f92c,2019-09-27,165,997286.0,99999.0,70.3,22.0,NaN,0.0,1017.1,22.0,NaN,0.0,NaN,0.0,0.0,22.0,NaN,NaN,79.3,*,63.7,*,0.0,I,NaN,0.0,4


In [11]:
# find the maximum date of F2019, make sure that it is less than or equal to 2019/11/24

Y2019.date.max()


Timestamp('2019-11-22 00:00:00')

In [12]:
# find the minimum date of F2019, make sure that it is greater than or equal to 2019/8/26

Y2019.date.min()


Timestamp('2019-08-26 00:00:00')

In [13]:
# display the unique values of dow in F2019 (use the .unique() method on dow column)
# make sure there are only M-F (e.g. 0-4)

Y2019.dow.unique()


array([0, 1, 2, 3, 4])

In [14]:
# compute the mean visits

Y2019.visits.mean()

137.73846153846154

In [15]:
# look up the total number of temple students in 2019 and compute the percentage the above average represents

138/38794*100


0.35572511213074187

**Edit this cell**
The average number of visitors in the sample is **138**, this is approximately **36**\% of the Temple student population.

**Edit this cell**

Before running a regression to document the relationship between temperature and the sampled visitors, answer the following.

The logical dependent variable should be **visits**, the logical indepdent (aka explanatory) variable should be **temperature** because **temperaturre** should cause changes in **visits** and not the other way around.


Consequently, the following regression equation describes the relationship between temperature and foot traffic (change the $x$ and $y$ to the approriate variable names):

$$
y = \beta_0 +\beta_1x + e 
$$


In [17]:
# run the above regression using statsmodels
# call the resulting regression results object the variable "res"

res = smf.ols('visits~temp', data=Y2019).fit()

In [18]:
# print the regression summary table (e.g. print(res.summary()))

print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                 visits   R-squared:                       0.468
Model:                            OLS   Adj. R-squared:                  0.459
Method:                 Least Squares   F-statistic:                     55.33
Date:                Wed, 20 Oct 2021   Prob (F-statistic):           3.43e-10
Time:                        00:10:10   Log-Likelihood:                -285.36
No. Observations:                  65   AIC:                             574.7
Df Residuals:                      63   BIC:                             579.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     40.8564     13.255      3.082      0.0

**Edit this cell**

The regression's R$^2$ is **0.468**, this suggests that **46.8**\% of the variation in **visits** is explained by **temperature**.

The intercept ($\beta_0$) is **40.8564**, this means that when **temperature** is 0 **(fill in appropriate units of measure)**, **visits** is on average **40.8564**. This is statistically **significant**, meaning that it is different from **0** in the population.

The coefficient ($\beta_1$) for **temperature** is **1.5446**. This means that for every **one degree** increase in **temperature**, **number of visits** **increases** by **1.5446**. This is statistically **significant**, meaning that it is different from **0** in the population. In other words, we can reject the null hypothesis that there is no effect of temperature on campus foot traffic.

# 2. Avocado price and demand (5 points) (multiple regression)

For this exercise, use our familiar avocado dataset.

For the following cities, run a multiple regression that investigates the relationship between demand and price while simultaneously accounting for the effect of regions.

Use only the conventional avocado data from the following cities:

```
cities = ['Philadelphia', 'NewYork', 'BaltimoreWashington', 'Boston', 'Chicago', 'SanFrancisco']
```


In [22]:
# read in the dataset, call it avocado
avocado = pd.read_csv(fpath + 'avocado.csv', index_col = 0)

cities = ['Philadelphia', 'NewYork', 'BaltimoreWashington', 'Boston', 'Chicago', 'SanFrancisco']


In [24]:
# select only the rows corresponding to the cities listed above
# call the resulting dataframe avocado (replace avocado with the result of this subset selection)
# use .isin(...) method (see notes from module 4 bar plot to find an example of this)
# also make sure to select only the data for the conventional avocado type

avocado = avocado.loc[avocado.region.isin(cities)&(avocado.type == 'conventional')]


In [25]:
# show the unique regions in the resulting dataframe 
# make sure we have only the ones we wanted to select from above

avocado.region.unique()


array(['BaltimoreWashington', 'Boston', 'Chicago', 'NewYork',
       'Philadelphia', 'SanFrancisco'], dtype=object)

In [26]:
# show the unique types in the resulting dataframe 
# make sure we have only conventional

avocado.type.unique()


array(['conventional'], dtype=object)

**edit this cell**
To model the demand of avocaldos, we want to run the following regression:

$$
Demand = \beta_0 + \beta_1 Price +\beta_2 Boston +\beta_3Chicago +\beta_4NewYork + \beta_5 Philadelphia + \beta_6 SanFrancisco +e
$$

In this equation (given the cities we've selected for the dataset), $\beta_0$ represents the average demand in **____________** when price is \$ $0$.



In [27]:
# since statsmodels expects variable names to not have spaces
# use a list comprehension to replace  spaces with underscores
# make sure you understand how/why this works:
avocado.columns = [c.replace(' ', '_') for c in avocado.columns]

In [28]:
# take a look at the resulting dataframe

avocado.head()


,Date,AveragePrice,Total_Volume,4046,4225,4770,Total_Bags,Small_Bags,Large_Bags,XLarge_Bags,type,year,region
104,2015-12-27,1.17,596819.40,40450.49,394104.02,17353.79,144911.10,142543.88,2367.22,0.0,conventional,2015,BaltimoreWashington
105,2015-12-20,1.11,642682.40,44741.37,434590.82,19112.14,144238.07,141751.91,2486.16,0.0,conventional,2015,BaltimoreWashington
106,2015-12-13,1.15,619509.33,44400.26,399851.59,24291.20,150966.28,149070.40,1895.88,0.0,conventional,2015,BaltimoreWashington
107,2015-12-06,1.04,649141.25,51129.29,453586.50,25760.49,118664.97,117112.77,1552.20,0.0,conventional,2015,BaltimoreWashington
108,2015-11-29,1.16,545800.05,41028.15,379156.09,20147.53,105468.28,103615.95,1852.33,0.0,conventional,2015,BaltimoreWashington


In [33]:
# run the regression using statsmodels
# call the result object res
# for demand, use 'Total Bags'

res = smf.ols('Total_Bags~AveragePrice+C(region)', data=avocado).fit()

In [34]:
# print the regression summary table

print(res.summary())

                            OLS Regression Results                            
Dep. Variable:             Total_Bags   R-squared:                       0.765
Model:                            OLS   Adj. R-squared:                  0.764
Method:                 Least Squares   F-statistic:                     546.7
Date:                Wed, 20 Oct 2021   Prob (F-statistic):          1.30e-312
Time:                        00:25:22   Log-Likelihood:                -12529.
No. Observations:                1014   AIC:                         2.507e+04
Df Residuals:                    1007   BIC:                         2.511e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept             

0.765**Edit this cell**
1. The R$^2$ is **0.765**, this means that **76.5**\% of the variation in demand is explained by the model.
1. The intercept ($\beta_0$) is **1.847e+05** and is statistically **significant**. This is the average demand in **BaltimoreWashinton** when price is \$0.
1. The coefficient for Boston is **-8.097e+04**, which means that the average demand in Boston is **8.097e+04** **fewer** bags than in **Baltimore**. This difference is statistically **significant**
1. The coefficient for price is **8121.8105**, which means that when price increases by \$1 in any region, demand increases by on average **8121.8105** bags. This effect is statistically **insignificant** which means that there is enough evidence to show that price **does not** have an **positive** impact on demand.
1. The region with the highest demand is **New York**.

# 3. Putting what we've learned all together (the power of programming) (2 points - extra point for bonus, try your best - there may or may not be a prize involved)

In this exercise, you will run one regression for each region and each avocado type, i.e. one regression of total bags sold on price for Philly conventional, one for Philly organic, one for NY conventional, one for NY organic, and so on... (think loop). 

You will then visualize the coefficients for conventional and organic price by region on a scatter plot (each dot's (x, y) coordinates = one region's (price coefficient for conventional, price coefficient for organic). 

To do this break down the problem to the following components (and then put it all together):

* Suppose you have selected a region $r$ and avocado type $t$, how would you run the regression?
* How can you extract just the price coefficient given a regression result object `res`?
* For some region, `r`, and some avocado type `t`, how would you select only the rows pertaining to region `r` and type `t`?
* How can you loop through regions and types? (Need 2 nested "for" loops, one inside the other)
* Suppose you have named the conventional and organic price coefficients for region `r` as `bConv` and `bOrg`, how would you create a list that looks like: `[region, conventional coeff, organic coefficient]`?
* Given a list `results`, how would you *append* the list above to `results`? i.e. make keep adding to `results`: `[[region1, conventional coeff1, organic coeff1], [region2, conventional coeff2, organic coeff2], [region3, conventional coeff3, organic coeff3],...]`
* Once you've run every regression and have a complete `results` list, convert it to a dataframe using `df = pd.DataFrame(results)`, give it some column names.

The final code might have a structure similar to the pseudo code below:

```
results = list()
for r in ##all the unique regions##:
    for t in ##all the unique types##:
        temp = # Select the appropriate rows of df
        # run the regression using temp
        if ##this iteration represents conventional avocados##:
            ##extract the price parameter assign it to the variable bConv##
        else:
            ##extract the price parameter and name it bOrg##
    ##append the results with newest [region, conventional price coeff, organic price coefficient]##
df = pd.DataFrame(results) # convert to dataframe
df.columns = ['region', 'beta_conv', 'beta_org'] # rename dataframe columns
```

The resulting dataframe of coefficients should look something like:

| region | beta_conv | beta_org |
|-|-|-|
| Albany |8.579340e+02	 |-3168.491384|
| Atlanta| -3.377365e+05 | -6352.937420 |
| BaltimoreWashington | -3.251570e+05 | -15537.003211 |
|...|...|...|...|

(note numbers above are illustrative, not necessarily representative of actual results)


In [54]:
# first re-read the original avocado dataframe to use for this problem

avocado = pd.read_csv(fpath + 'avocado.csv', index_col = 0)

avocado.head(10)

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany
5,2015-11-22,1.26,55979.78,1184.27,48067.99,43.61,6683.91,6556.47,127.44,0.0,conventional,2015,Albany
6,2015-11-15,0.99,83453.76,1368.92,73672.72,93.26,8318.86,8196.81,122.05,0.0,conventional,2015,Albany
7,2015-11-08,0.98,109428.33,703.75,101815.36,80.00,6829.22,6266.85,562.37,0.0,conventional,2015,Albany
8,2015-11-01,1.02,99811.42,1022.15,87315.57,85.34,11388.36,11104.53,283.83,0.0,conventional,2015,Albany
9,2015-10-25,1.07,74338.76,842.40,64757.44,113.00,8625.92,8061.47,564.45,0.0,conventional,2015,Albany


In [39]:
# rename columns to get rid of spaces

avocado.columns = [c.replace(' ', '_') for c in avocado.columns]

In [55]:
# run all the regressions
# create a dataframe with region, beta_conv, beta_org as the 3 columns 
# using the regression results

res = smf.ols('region~beta_conv+C(beta_org)', data=avocado).fit()

PatsyError: ignored

In [53]:
res = smf.ols('conventional~organic+C(region)', data=avocado).fit()

PatsyError: ignored

In [ ]:
# Get rid of the TotalUS region from the regression results dataframe



In [ ]:
# show the full resulting table



In [68]:
# make scatter plot of the coefficients
# change aesthetics so the dots are not a single blob

plt.figure(figsize = (10,8))
plt.scatter(avocado.type(conventional), 
            avocado.type(organic), 
            alpha = .5,  
            cmap = 'RdYlGn_r')
plt.xlabel('Conventional')
plt.ylabel('Organic')


NameError: ignored

<Figure size 720x576 with 0 Axes>

In [ ]:
# which regions have upward sloping demand curve for organic (as price increases, so did demand)?
# use code to show these rows of df



In [ ]:
# which regions have upward sloping demand curve for conventional (as price increases, so did demand)?
# use code to show these rows of df



Why might the relationship between price and demand be upward sloping instead of the expected downward sloping demand curve theorized in economics? i.e. the law of demand, when price increases, demand decreases. 

**Your answer here**

The product could more elasticity than expected, and depending on the region, consumers are more willing to pay more for their avocados. (just guessing)